In [26]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import block
import ecc
import helper
import script
import tx

### Exercise 1

Write the `is_coinbase` method of the `Tx` class.

#### Make [this test](/edit/code-ch09/tx.py) pass: `tx.py:TxTest:test_is_coinbase`



`def is_coinbase(self):`
        `'''Returns whether this transaction is a coinbase transaction or not'''`
        # check that there is exactly 1 input
        if len(self.tx_ins) != 1:
        # grab the first input
            return False
        
        first_input = self.tx_ins[0]
        
        # check that first input prev_tx is b'\x00' * 32 bytes
        if first_input.prev_tx != b'\x00' * 32:
            return False
        
        # check that first input prev_index is 0xffffffff
        if first_input.prev_index != 0xffffffff:
            return False
        return True

In [27]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_is_coinbase"))

.
----------------------------------------------------------------------
Ran 1 test in 0.054s

OK


In [28]:
from io import BytesIO
from script import Script
stream = BytesIO(bytes.fromhex('4d04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73'))
s = Script.parse(stream)
print(s.cmds[2])

b'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks'


In [29]:
from io import BytesIO
from script import Script
from helper import little_endian_to_int
stream = BytesIO(bytes.fromhex('5e03d71b07254d696e656420627920416e74506f6f6c20626a31312f4542312f4144362f43205914293101fabe6d6d678e2c8c34afc36896e7d9402824ed38e856676ee94bfdb0c6c4bcd8b2e5666a0400000000000000c7270000a5e00e00'))
script_sig = Script.parse(stream)
print(little_endian_to_int(script_sig.cmds[0]))

465879


### Exercise 2

Write the `coinbase_height` method for the `Tx` class.

#### Make [this test](/edit/code-ch09/tx.py) pass: `tx.py:TxTest:test_coinbase_height`

In [30]:
    def coinbase_height(self):
        '''Returns the height of the block this coinbase transaction is in
        Returns None if this transaction is not a coinbase transaction
        '''
        # if this is NOT a coinbase transaction, return None
        if not self.is_coinbase():
            return None
        
        # grab the first cmd
        element = self.tx_ins[0].script_sig.cmds[0]
        
        # convert the cmd from little endian to int
        return little_endian_to_int(element)


In [31]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_coinbase_height"))

.
----------------------------------------------------------------------
Ran 1 test in 0.013s

OK


In [32]:
from helper import hash256
block_hash = hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'))[::-1]
block_id = block_hash.hex()
print(block_id)

0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523


### Exercise 3

Write the `parse` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_parse`

In [33]:
@classmethod
def parse(cls, s):
        '''Takes a byte stream and parses the tx_input at the start
        return a TxIn object
        '''
        # prev_tx is 32 bytes, little endian
        prev_tx = s.read(32)[::-1]
        # prev_index is an integer in 4 bytes, little endian
        prev_index = little_endian_to_int(s.read(4))
        # use Script.parse to get the ScriptSig
        script_sig = Script.parse(s)
        # sequence is an integer in 4 bytes, little-endian
        sequence = little_endian_to_int(s.read(4))
        # return an instance of the class (see __init__ for args)
        return cls(prev_tx, prev_index, script_sig, sequence)


In [34]:
# Exercise 3

reload(block)
run(block.BlockTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


# 7 Stars Seven Serialize

### Exercise 4

Write the `serialize` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_serialize`

In [35]:
def serialize(self):
        '''Returns the 80 byte block header'''
        # version - 4 bytes, little endian
        result = int_to_little_endian(self.version, 4)
        # prev_block - 32 bytes, little endian
        result+= self.prev_block[::-1]
        # merkle_root - 32 bytes, little endian
        result+= self.merkle_root[::-1]
        # timestamp - 4 bytes, little endian
        result+= int_to_little_endian(self.timestamp, 4)
        # bits - 4 bytes
        result+= self.bits
        # nonce - 4 bytes
        result+= self.nonce
        return result

In [36]:
# Exercise 4

reload(block)
run(block.BlockTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


# Check Version = Conversion

### Exercise 5

Write the `hash` for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_hash`

In [37]:
def hash(self):
        '''Returns the hash256 interpreted little endian of the block'''
        # serialize
        s = self.serialize()
        # hash256
        sha = hash256(s)
        # reverse
        return sha[::-1]

In [38]:
# Exercise 5

reload(block)
run(block.BlockTest("test_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


In [39]:
from io import BytesIO
from block import Block
b = Block.parse(BytesIO(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d')))
print('BIP9: {}'.format(b.version >> 29 == 0b001))
print('BIP91: {}'.format(b.version >> 4 & 1 == 1))
print('BIP141: {}'.format(b.version >> 1 & 1 == 1))

BIP9: True
BIP91: False
BIP141: True


### Exercise 6

Write the `bip9` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip9`

In [40]:
def bip9(self):
        '''Returns whether this block is signaling readiness for BIP9'''
        # BIP9 is signalled if the top 3 bits are 001
        # remember version is 32 bytes so right shift 29 (>> 29) and see if
        return self.version >> 29==0b001# that is 001

In [41]:
# Exercise 6

reload(block)
run(block.BlockTest("test_bip9"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 7

Write the `bip91` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip91`

In [42]:

    def bip91(self):
        '''Returns whether this block is signaling readiness for BIP91'''
        # BIP91 is signalled if the 5th bit from the right is 1
        # shift 4 bits to the right and see if the last bit is 1
        return self.version >> 4 & 1==1


In [43]:
# Exercise 7

reload(block)
run(block.BlockTest("test_bip91"))

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


### Exercise 8

Write the `bip141` method for the `Block` class.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_bip141`

In [44]:

    def bip141(self):
        '''Returns whether this block is signaling readiness for BIP141'''
        # BIP91 is signalled if the 2nd bit from the right is 1
        # shift 1 bit to the right and see if the last bit is 1
        return self.version >> 1 & 1==1

In [45]:
# Exercise 8

reload(block)
run(block.BlockTest("test_bip141"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [46]:
from helper import hash256
block_id = hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'))[::-1]
print('{}'.format(block_id.hex()).zfill(64))

0000000000000000007e9e4c586439b0cdbe13b1370bdd9435d76a644d047523


# Target - Goal Setting?

In [47]:
from helper import little_endian_to_int
bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient * 256**(exponent - 3)
print('{:x}'.format(target).zfill(64))

0000000000000000013ce9000000000000000000000000000000000000000000


In [48]:
from helper import little_endian_to_int
proof = little_endian_to_int(hash256(bytes.fromhex('020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d')))
print(proof < target)

True


### Exercise 9

Write the `bits_to_target` function in `helper.py`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_target`

In [53]:
def difficulty(self):
        '''Returns the block difficulty based on the bits'''
        # note difficulty is (target of lowest difficulty) / (self's target)
        lowest = 0xffff * 256**(0x1d-3)
        
        # lowest difficulty has bits that equal 0xffff001d
        return lowest / self.target()

In [54]:

def bits_to_target(bits):
    '''Turns bits into a target (large 256-bit integer)'''
    # last byte is exponent
    exponent = bits[-1]
    # the first three bytes are the coefficient in little endian
    coefficient = little_endian_to_int(bits[:-1])
    
    # the formula is:
    target = coefficient * 256**(exponent-3) # coefficient * 256**(exponent-3)
    
    return target

In [55]:
# Exercise 9

reload(block)
run(block.BlockTest("test_target"))

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


In [56]:
from helper import little_endian_to_int
bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient*256**(exponent-3)
difficulty = 0xffff * 256**(0x1d-3) / target
print(difficulty)

888171856257.3206


### Exercise 10

Write the `difficulty` method for `Block`

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_difficulty`

In [57]:
# Exercise 10

reload(block)
run(block.BlockTest("test_difficulty"))

.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


### Exercise 11

Write the `check_pow` method for `Block`.

#### Make [this test](/edit/code-ch09/block.py) pass: `block.py:BlockTest:test_check_pow`

In [60]:
# Exercise 11

reload(block)
run(block.BlockTest("test_check_pow"))

.
----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


In [62]:
from block import Block
from helper import TWO_WEEKS
last_block = Block.parse(BytesIO(bytes.fromhex('00000020fdf740b0e49cf75bb3d5168fb3586f7613dcc5cd89675b0100000000000000002e37b144c0baced07eb7e7b64da916cd3121f2427005551aeb0ec6a6402ac7d7f0e4235954d801187f5da9f5')))
first_block = Block.parse(BytesIO(bytes.fromhex('000000201ecd89664fd205a37566e694269ed76e425803003628ab010000000000000000bfcade29d080d9aae8fd461254b041805ae442749f2a40100440fc0e3d5868e55019345954d80118a1721b2e')))
time_differential = last_block.timestamp - first_block.timestamp
if time_differential > TWO_WEEKS * 4:
    time_differential = TWO_WEEKS * 4
if time_differential < TWO_WEEKS // 4:
    time_differential = TWO_WEEKS // 4
new_target = last_block.target() * time_differential // TWO_WEEKS
print('{:x}'.format(new_target).zfill(64))

0000000000000000007615000000000000000000000000000000000000000000


### Exercise 12

Calculate the new bits given the first and last blocks of this 2016 block difficulty adjustment period:

Block 471744:

```
000000203471101bbda3fe307664b3283a9ef0e97d9a38a7eacd8800000000000000000010c8ab
a8479bbaa5e0848152fd3c2289ca50e1c3e58c9a4faaafbdf5803c5448ddb845597e8b0118e43a
81d3
```

Block 473759:

```
02000020f1472d9db4b563c35f97c428ac903f23b7fc055d1cfc26000000000000000000b3f449
fcbe1bc4cfbcb8283a0d2c037f961a3fdf2b8bedc144973735eea707e1264258597e8b0118e5f0
0474
```

In [72]:
# Exercise 12

from block import Block
from helper import target_to_bits, TWO_WEEKS

block1_hex = '000000203471101bbda3fe307664b3283a9ef0e97d9a38a7eacd8800000000000000000010c8aba8479bbaa5e0848152fd3c2289ca50e1c3e58c9a4faaafbdf5803c5448ddb845597e8b0118e43a81d3'
block2_hex = '02000020f1472d9db4b563c35f97c428ac903f23b7fc055d1cfc26000000000000000000b3f449fcbe1bc4cfbcb8283a0d2c037f961a3fdf2b8bedc144973735eea707e1264258597e8b0118e5f00474'

# parse both blocks
last_block = Block.parse(BytesIO(bytes.fromhex(block1_hex)))
first_block = Block.parse(BytesIO(bytes.fromhex(block2_hex)))


# get the time differential
time_differential = last_block.timestamp - first_block.timestamp

# if the differential > 8 weeks, set to 8 weeks
if time_differential  > TWO_WEEKS * 4:
    time_differential = TWO_WEEKS *4

# if the differential < 1/2 week, set to 1/2 week
if time_differential  < TWO_WEEKS // 4:
    time_differential = TWO_WEEKS //4

# new target is last target * differential / 2 weeks
new_target = last_block.target() * time_differential // TWO_WEEKS

# convert new target to bits
new_bits = target_to_bits(new_target)

# print the new bits hex
print(new_bits.hex())


80df6217


### Exercise 13

Write the `calculate_new_bits` function in `helper.py`

#### Make [this test](/edit/code-ch09/helper.py) pass: `helper.py:HelperTest:test_calculate_new_bits`

In [71]:
# Exercise 13

reload(helper)
run(helper.HelperTest("test_calculate_new_bits"))

.
----------------------------------------------------------------------
Ran 1 test in 0.010s

OK
